In [ ]:
from pprint import pprint

from utils.file_utils import load_json, dump_json
from utils.common import compute_rtts_per_dst_src, round_based_algorithm
from default import PROBES_FILE, GREEDY_PROBES_FILE, PROBES_TO_ANCHORS_PING_TABLE, ROUND_BASED_ALGORITHM_FILE


TIER1_VPS = [10, 100, 300, 500, 1000]

## Step X

In [ ]:
probes = load_json(PROBES_FILE)

greedy_probes = load_json(GREEDY_PROBES_FILE)

In [ ]:
asn_per_vp_ip = {}
vp_coordinates_per_ip = {}

for probe in probes:
    if "address_v4" in probe and "geometry" in probe and "coordinates" in probe["geometry"]:
        ip_v4_address = probe["address_v4"]
        if ip_v4_address is None:
            continue
        long, lat = probe["geometry"]["coordinates"]
        asn_v4 = probe["asn_v4"]
        asn_per_vp_ip[ip_v4_address] = asn_v4
        vp_coordinates_per_ip[ip_v4_address] = lat, long


In [ ]:
filter = ""
# clickhouse is required here
rtt_per_srcs_dst = compute_rtts_per_dst_src(PROBES_TO_ANCHORS_PING_TABLE, filter, threshold=100)

error_cdf_per_tier1_vps = {}
for tier1_vps in TIER1_VPS:
    pprint(f"Using {tier1_vps} tier1_vps")
    error_cdf = round_based_algorithm(greedy_probes, rtt_per_srcs_dst, vp_coordinates_per_ip,
                                        asn_per_vp_ip,
                                        tier1_vps,
                                        threshold=40)
    error_cdf_per_tier1_vps[tier1_vps] = error_cdf
    
dump_json(error_cdf_per_tier1_vps, ROUND_BASED_ALGORITHM_FILE)